In [1]:
import os
import pandas as pd

In [2]:
while "notebooks" in os.getcwd():
    os.chdir("../")

In [3]:
os.chdir("src")

In [4]:
from decision_tree.tree import parse_xgboost_dump, TreeEnsemble

In [5]:
trees = parse_xgboost_dump("../models/heloc-scaled-gbdt_dumped.txt")

In [6]:
some_trees = TreeEnsemble(trees.trees)
#some_trees = TreeEnsemble(trees.trees[0:10])

In [7]:
df = pd.read_csv("../data/heloc-scaled-test.csv")

In [8]:
df

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance,RiskPerformance
0,-0.214900,1.699313,-0.056723,2.459516,-0.363267,1.201020,1.712719,-1.564641,-1.466360,-3.498869,...,-0.488720,-0.211494,-0.187532,-1.005586,0.090931,-0.702250,-0.955119,-0.062510,-1.379531,0.0
1,1.716386,-0.361454,-0.280331,0.499039,-0.185539,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,0.256357,-0.668264,-0.652592,-0.866162,-0.122189,-0.029287,-0.307994,-0.062510,0.391373,0.0
2,0.903213,0.638018,-0.578475,-0.027656,2.036062,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,1.852951,1.615589,1.672709,-1.075297,1.598802,-0.365768,-0.307994,-0.740481,-1.879017,1.0
3,-0.011607,-0.309935,-0.503939,-1.139568,-0.896451,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,-0.701599,0.245277,0.277528,0.946344,1.446950,0.307195,-0.307994,0.615461,0.981674,0.0
4,0.394979,0.586499,-0.503939,0.323474,1.680606,1.201020,0.658902,-0.119124,0.681985,0.147707,...,-0.062962,-0.668264,-0.652592,-0.238756,-0.096058,1.316640,-0.307994,-0.062510,-0.743822,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,-0.926427,-1.731864,-0.429403,-1.871090,-0.896451,-0.476838,-0.394914,-0.884398,-1.495790,-1.067818,...,-0.169401,-0.668264,-0.652592,-0.761594,0.090931,-0.029287,-0.955119,-0.740481,-0.743822,0.0
1971,-0.621487,-0.608746,-0.056723,-0.964003,-1.429635,0.362091,0.658902,-0.799368,-0.377473,0.147707,...,-0.275841,0.702048,0.742588,0.493218,0.090931,-0.365768,0.056953,-0.062510,1.526568,0.0
1972,0.191686,-0.598442,-0.429403,-0.612873,-0.096675,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,1.852951,1.158818,1.207648,-0.308468,-1.438241,0.307195,0.339132,0.615461,-0.425967,0.0
1973,1.309799,0.205257,-0.578475,-0.964003,3.635614,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,-0.701599,0.245277,0.277528,-1.214721,-0.096058,-0.029287,0.339132,-0.740481,-2.060648,1.0


In [9]:
some_trees.eval(df.iloc[25, :-1])

-0.3071252657664999

In [10]:
x = df.iloc[1, :-1]

In [11]:
x

ExternalRiskEstimate                  1.716386
MSinceOldestTradeOpen                -0.361454
MSinceMostRecentTradeOpen            -0.280331
AverageMInFile                        0.499039
NumSatisfactoryTrades                -0.185539
NumTrades60Ever2DerogPubRec          -0.476838
NumTrades90Ever2DerogPubRec          -0.394914
PercentTradesNeverDelq                0.646149
MSinceMostRecentDelq                  0.946849
MaxDelq2PublicRecLast12M              0.755470
MaxDelqEver                           0.876990
NumTotalTrades                       -0.280830
NumTradesOpeninLast12M               -0.465434
PercentInstallTrades                 -0.146751
MSinceMostRecentInqexcl7days          0.256357
NumInqLast6M                         -0.668264
NumInqLast6Mexcl7days                -0.652592
NetFractionRevolvingBurden           -0.866162
NetFractionInstallBurden             -0.122189
NumRevolvingTradesWBalance           -0.029287
NumInstallTradesWBalance             -0.307994
NumBank2NatlT

In [12]:
y = some_trees.eval(x)

In [13]:
y

2.1090505724301014

## Porównanko ewaluacji na pierwszych 20 datapointach

In [14]:
import xgboost as xgb
from pathlib import Path


In [15]:
model_path = Path("../models")
model_name = "heloc-scaled-gbdt"
gbdt_model = xgb.Booster()
gbdt_model.load_model(model_path / f"{model_name}_saved.json")


In [16]:
true_labels = df.iloc[:20,-1]


In [17]:
adam_trees_eval = [some_trees.eval(df.iloc[i, :-1]) for i in range(20)]


In [18]:
x_dmatrix = xgb.DMatrix(df.iloc[:20, :-1])
actual_xgb_model_eval = gbdt_model.predict(x_dmatrix)


In [19]:
from numpy import exp
adam_sigmoid = map(lambda x: 1 / (1 + exp(-x)), adam_trees_eval)

In [20]:
pd.DataFrame(zip(true_labels, adam_trees_eval, adam_sigmoid, actual_xgb_model_eval),
             columns=["RiskPerformance", "Eval by Adam Trees", "Eval by Adam Trees + sigmoid", "Actual eval by XGB model"])


,RiskPerformance,Eval by Adam Trees,Eval by Adam Trees + sigmoid,Actual eval by XGB model
0,0.0,-3.356478,0.033684,0.033684
1,0.0,2.109051,0.891780,0.891780
2,1.0,1.555286,0.825676,0.825676
3,0.0,-1.179208,0.235195,0.235195
4,1.0,4.007997,0.982155,0.982154
5,1.0,0.571139,0.639026,0.639026
6,0.0,-0.826656,0.304353,0.304353
7,0.0,-0.583389,0.358153,0.358153
8,1.0,-0.207345,0.448349,0.448349
9,0.0,-1.910231,0.128955,0.128955


Widzimy że wyniki z dwóch ostatnich kolumn są różne, a powinny być dokładnie takie same.

In [21]:
from decision_tree.prediction_gap import NormalPredictionGap

In [22]:
pred_gap = NormalPredictionGap(0.3)

Powinno wyjść 0

In [23]:
pred_gap.prediction_gap_fixed(some_trees, x, y, set([]))

3.802513859341161e-15

### PGI dla każdego pojedynczego featura i wybranego x: kwadrat ogólny, kwadrat szybki, moduł szybki

In [24]:
# %%time
# for column in df.columns:
#     print("%-40s %.15f %.15f %.15f" % (column,
#         pred_gap.prediction_gap_fixed(some_trees, x, y, set([column])),
#         pred_gap.prediction_gap_single_squared(some_trees, x, column, y),
#         pred_gap.prediction_gap_single_abs(some_trees, x, column, y)
#     ))

In [25]:
%%time
for column in df.columns[:-1]:
    print("%-40s %.15f %.15f" % (column,
        # pred_gap.prediction_gap_fixed(some_trees, x, y, set([column])),
        pred_gap.prediction_gap_single_squared(some_trees, x, column, y),
        pred_gap.prediction_gap_single_abs(some_trees, x, column, y)
    ))

ExternalRiskEstimate                     0.000010803060203 0.000175981743570
MSinceOldestTradeOpen                    0.002138098476941 0.018216024914377
MSinceMostRecentTradeOpen                0.000000000000000 0.000000000000000
AverageMInFile                           0.000698272040092 0.012798448336813
NumSatisfactoryTrades                    0.000634618798494 0.005662415651700
NumTrades60Ever2DerogPubRec              0.000000000000000 0.000000000000000
NumTrades90Ever2DerogPubRec              0.000000000000000 0.000000000000000
PercentTradesNeverDelq                   0.004048991536556 0.019584189727122
MSinceMostRecentDelq                     0.000000001309236 0.000000004411294
MaxDelq2PublicRecLast12M                 0.000000000000000 0.000000000000000
MaxDelqEver                              0.000000000000000 0.000000000000000
NumTotalTrades                           0.000045261998306 0.003434307043302
NumTradesOpeninLast12M                   0.000000000000000 0.000000000000000

In [26]:
x = df.iloc[1, :-1]
y = some_trees.eval(x)


In [27]:
%%time
for column in df.columns[:-1]:
    print("%-40s %.15f" % (column,
        pred_gap.prediction_gap_fixed(some_trees, x, y, set([column]))
        #pred_gap.prediction_gap_single_squared(some_trees, x, column, y),
        #pred_gap.prediction_gap_single_abs(some_trees, x, column, y)
    ))

ExternalRiskEstimate                     0.000010803060150
MSinceOldestTradeOpen                    0.002138098476932
MSinceMostRecentTradeOpen                0.000000000000004
AverageMInFile                           0.000698272040089
NumSatisfactoryTrades                    0.000634618798492
NumTrades60Ever2DerogPubRec              0.000000000000004
NumTrades90Ever2DerogPubRec              0.000000000000004
PercentTradesNeverDelq                   0.004048991536534
MSinceMostRecentDelq                     0.000000001309238
MaxDelq2PublicRecLast12M                 0.000000000000004
MaxDelqEver                              0.000000000000004
NumTotalTrades                           0.000045261998304
NumTradesOpeninLast12M                   0.000000000000004
PercentInstallTrades                     0.000111027768562
MSinceMostRecentInqexcl7days             0.007441254238171
NumInqLast6M                             0.002580418430898
NumInqLast6Mexcl7days                    0.0000476162516

### Poniżej przykład przeliczony na kartce (z innym rozkładem co prawda)

In [28]:
from decision_tree.tree import Split, Leaf

In [29]:
test_tree = TreeEnsemble([Split("dupa", 0, Split("dupa", -0.5, Leaf(-1), Leaf(5), Leaf(0)), Leaf(1), Leaf(0))])

In [30]:
x = {'dupa': 0.1}
y = test_tree.eval(x)

In [31]:
pred_gap.prediction_gap_fixed(test_tree, x, y, set(["dupa"]))

5.638059859512101

In [32]:
pred_gap.prediction_gap_single_squared(test_tree, x, "dupa", y)

5.638059859530067

### Global prediction-gap dla każdego feature'a przez 100 pierwszych punktow

Single squared liczy się 4 minuty dla 100 datapoints, single absolute również. Trochę głupio że liczymy prediction gap dla RiskPerformance, to jest wartość którą model ma przewidzieć, a nie jeden z feature'ów. Dobrze że wychodzi zero dla RiskPerformance. Próbowałem dać `x = df.iloc[i, :-1]` żeby wyłączyć RiskPerformance z iksa, ale wtedy wywala się.

In [33]:
%%time
results_sqr = []
for column in df.columns[:-1]:
    glob = 0.0
    for i in range(100):
        x = df.iloc[i, :-1]
        glob += pred_gap.prediction_gap_single_squared(some_trees, x, column, some_trees.eval(x))
    glob /= df.shape[0]
    results_sqr.append((column, glob))
    print("%-40s %.15f" % (column, glob))


ExternalRiskEstimate                     0.002211830895537
MSinceOldestTradeOpen                    0.000152114035040
MSinceMostRecentTradeOpen                0.000003313913782
AverageMInFile                           0.000228543563430
NumSatisfactoryTrades                    0.000282277125343
NumTrades60Ever2DerogPubRec              0.000000431586188
NumTrades90Ever2DerogPubRec              0.000000000000053
PercentTradesNeverDelq                   0.000081791716738
MSinceMostRecentDelq                     0.000719393254660
MaxDelq2PublicRecLast12M                 0.000014157339063
MaxDelqEver                              0.000021993845171
NumTotalTrades                           0.000063863154940
NumTradesOpeninLast12M                   0.000021986676333
PercentInstallTrades                     0.000169418590392
MSinceMostRecentInqexcl7days             0.006761520332627
NumInqLast6M                             0.000073318333217
NumInqLast6Mexcl7days                    0.0000087294697

In [34]:
%%time
results_abs = []
for column in df.columns[:-1]:
    glob = 0.0
    for i in range(100):
        x = df.iloc[i, :-1]
        glob += pred_gap.prediction_gap_single_abs(some_trees, x, column, some_trees.eval(x))
    glob /= df.shape[0]
    results_abs.append((column, glob))
    print("%-40s %.15f" % (column, glob))


ExternalRiskEstimate                     0.005155344089779
MSinceOldestTradeOpen                    0.000865388433487
MSinceMostRecentTradeOpen                0.000032876288498
AverageMInFile                           0.001472741899978
NumSatisfactoryTrades                    0.001225200420146
NumTrades60Ever2DerogPubRec              0.000006468758360
NumTrades90Ever2DerogPubRec              0.000000000001353
PercentTradesNeverDelq                   0.000665775688573
MSinceMostRecentDelq                     0.001767020920832
MaxDelq2PublicRecLast12M                 0.000108605650565
MaxDelqEver                              0.000090847599853
NumTotalTrades                           0.000628714902540
NumTradesOpeninLast12M                   0.000153915385540
PercentInstallTrades                     0.001115931729341
MSinceMostRecentInqexcl7days             0.009945055075049
NumInqLast6M                             0.000615469856763
NumInqLast6Mexcl7days                    0.0001250781928

In [35]:
df_results_sqr = pd.DataFrame(results_sqr, columns=["Feature", "PredGap"])
df_results_sqr.sort_values("PredGap", ascending=False)


,Feature,PredGap
18,NetFractionInstallBurden,1.464545e-01
14,MSinceMostRecentInqexcl7days,6.761520e-03
21,NumBank2NatlTradesWHighUtilization,6.455006e-03
0,ExternalRiskEstimate,2.211831e-03
8,MSinceMostRecentDelq,7.193933e-04
17,NetFractionRevolvingBurden,3.248936e-04
4,NumSatisfactoryTrades,2.822771e-04
3,AverageMInFile,2.285436e-04
13,PercentInstallTrades,1.694186e-04
1,MSinceOldestTradeOpen,1.521140e-04


In [36]:
df_results_abs = pd.DataFrame(results_abs, columns=["Feature", "PredGap"])
df_results_abs.sort_values("PredGap", ascending=False)


,Feature,PredGap
18,NetFractionInstallBurden,4.696939e-02
14,MSinceMostRecentInqexcl7days,9.945055e-03
21,NumBank2NatlTradesWHighUtilization,9.155635e-03
0,ExternalRiskEstimate,5.155344e-03
8,MSinceMostRecentDelq,1.767021e-03
17,NetFractionRevolvingBurden,1.700608e-03
3,AverageMInFile,1.472742e-03
4,NumSatisfactoryTrades,1.225200e-03
13,PercentInstallTrades,1.115932e-03
1,MSinceOldestTradeOpen,8.653884e-04


Widzimy drobne różnice pomiędzy kolejnością cech przy sortowaniu po pred gap kwadratowej a bezwzględnej. Jednakże sześć najważniejszych jest w dokładnie tej samej kolejności i żadna cecha nie przesuwa się o więcej niż 3 pozycje.

In [37]:
banzhaf_results = [5.3082e-01, 6.7819e-02, 1.0057e-03, 1.1615e-01, 7.9269e-02, 7.5800e-04,
        1.9180e-04, 4.1986e-02, 1.2749e-01, 3.8168e-03, 1.4541e-03, 3.7466e-02,
        1.3944e-03, 5.2826e-02, 2.1571e-01, 2.6424e-02, 7.8650e-03, 1.4590e-01,
        1.0155e+00, 6.1204e-03, 1.3284e-02, 1.1442e-01, 2.2595e-02]

In [38]:
df_results_banzhaf = pd.DataFrame(zip(df_results_abs.Feature, banzhaf_results),
                                  columns=["Feature", "banzhaf global importance"])

In [39]:
df_results_banzhaf


,Feature,banzhaf global importance
0,ExternalRiskEstimate,0.530820
1,MSinceOldestTradeOpen,0.067819
2,MSinceMostRecentTradeOpen,0.001006
3,AverageMInFile,0.116150
4,NumSatisfactoryTrades,0.079269
5,NumTrades60Ever2DerogPubRec,0.000758
6,NumTrades90Ever2DerogPubRec,0.000192
7,PercentTradesNeverDelq,0.041986
8,MSinceMostRecentDelq,0.127490
9,MaxDelq2PublicRecLast12M,0.003817


In [40]:
df_results_banzhaf.sort_values("banzhaf global importance", ascending=False)


,Feature,banzhaf global importance
18,NetFractionInstallBurden,1.015500
0,ExternalRiskEstimate,0.530820
14,MSinceMostRecentInqexcl7days,0.215710
17,NetFractionRevolvingBurden,0.145900
8,MSinceMostRecentDelq,0.127490
3,AverageMInFile,0.116150
21,NumBank2NatlTradesWHighUtilization,0.114420
4,NumSatisfactoryTrades,0.079269
1,MSinceOldestTradeOpen,0.067819
13,PercentInstallTrades,0.052826


Widzimy znaczne różnice pomiędzy tym, jak wygląda kolejność wyznaczona przez Banzhaf global importances a nasze single pred gap, jednakże nie są one zupełnie od czapy (oba zgadzają się że cechy 0, 3, 8, 14, 18, 21 należą do najważniejszych).

## Wyliczanie PGI przez samplowanie

In [41]:
from pathlib import Path
import xgboost as xgb
import numpy as np


In [42]:
model_path = Path("../models")
model_name = "heloc-scaled-gbdt"
gbdt_model = xgb.Booster()
gbdt_model.load_model(model_path / f"{model_name}_saved.json")


In [43]:
big_x = df.iloc[:100, :-1]
x_dmatrix = xgb.DMatrix(big_x)
f = gbdt_model.predict(x_dmatrix)


In [44]:
rng = np.random.default_rng(seed=2137)


In [45]:
banzhaf_top_four = {"NetFractionInstallBurden",
                    "ExternalRiskEstimate",
                    "MSinceMostRecentInqexcl7days",
                    "NetFractionRevolvingBurden"}

single_abs_predgap_top_four = {"PercentInstallTrades",
                               "MSinceMostRecentInqexcl7days",
                               "ExternalRiskEstimate",
                               "NumBank2NatlTradesWHighUtilization"}


In [46]:
banzhaf_bottom = set(df_results_banzhaf.sort_values("banzhaf global importance", ascending=False).iloc[-4:]["Feature"])
single_abs_predgap_bottom = set(df_results_abs.sort_values("PredGap", ascending=False).iloc[-5:-1]["Feature"])
display(banzhaf_bottom)
display(single_abs_predgap_bottom)
display(banzhaf_bottom & single_abs_predgap_bottom)

{'MSinceMostRecentTradeOpen',
 'NumTrades60Ever2DerogPubRec',
 'NumTrades90Ever2DerogPubRec',
 'NumTradesOpeninLast12M'}

{'MSinceMostRecentTradeOpen',
 'MaxDelq2PublicRecLast12M',
 'MaxDelqEver',
 'NumTrades60Ever2DerogPubRec'}

{'MSinceMostRecentTradeOpen', 'NumTrades60Ever2DerogPubRec'}

In [47]:
big_x

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,-0.214900,1.699313,-0.056723,2.459516,-0.363267,1.201020,1.712719,-1.564641,-1.466360,-3.498869,...,-1.310432,-0.488720,-0.211494,-0.187532,-1.005586,0.090931,-0.702250,-0.955119,-0.062510,-1.379531
1,1.716386,-0.361454,-0.280331,0.499039,-0.185539,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,-0.146751,0.256357,-0.668264,-0.652592,-0.866162,-0.122189,-0.029287,-0.307994,-0.062510,0.391373
2,0.903213,0.638018,-0.578475,-0.027656,2.036062,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,-0.811712,1.852951,1.615589,1.672709,-1.075297,1.598802,-0.365768,-0.307994,-0.740481,-1.879017
3,-0.011607,-0.309935,-0.503939,-1.139568,-0.896451,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,-0.423818,-0.701599,0.245277,0.277528,0.946344,1.446950,0.307195,-0.307994,0.615461,0.981674
4,0.394979,0.586499,-0.503939,0.323474,1.680606,1.201020,0.658902,-0.119124,0.681985,0.147707,...,-0.645472,-0.062962,-0.668264,-0.652592,-0.238756,-0.096058,1.316640,-0.307994,-0.062510,-0.743822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.598273,0.452549,0.166885,-1.812568,-1.518499,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,0.850689,0.894995,-0.668264,-0.652592,-1.214721,0.181515,-1.375213,-0.307994,-0.740481,-0.743822
96,-0.418194,0.040395,-0.354867,-0.817699,0.791966,-0.476838,-0.394914,0.391058,-1.289784,-1.067818,...,-0.645472,1.852951,-0.668264,-0.652592,-0.134188,1.042011,0.307195,0.986257,-0.062510,-0.970861
97,-1.434660,-1.845207,-0.578475,-2.075915,-1.518499,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,0.850689,1.852951,0.245277,0.277528,2.305724,0.383984,-1.038732,0.339132,-0.062510,1.526568
98,-0.113254,-0.361454,-0.205795,-0.700656,-0.363267,-0.476838,-0.394914,-0.289185,-0.024320,0.147707,...,1.848130,0.043478,-0.668264,-0.652592,-0.413035,0.839541,-0.365768,0.339132,-0.062510,1.526568


In [48]:
def normal_perturbation(x, mu, stddev):
    perturbed_x = x.copy()
    perturbed_x[list(perturbed_columns)] += rng.normal(loc=mu, scale=stddev,
                                                       size=(len(x), len(perturbed_columns)))
    x_dmatrix = xgb.DMatrix(perturbed_x)
    return gbdt_model.predict(x_dmatrix)
    

In [49]:
def prediction_gap_by_random_sampling(x, y, perturbed_columns: set,
                                      seed: int = 42, num_iter: int = 100):
    rng = np.random.default_rng(seed=seed)
    result = np.zeros(len(x))
    for i in range(num_iter):
        perturbed_y = normal_perturbation(x, 0.0, 1.0)
        result += np.abs(perturbed_y - y)
    result /= num_iter
    return np.mean(result)


## PGI results, 4 most important features are perturbed

The results are very close to each other, they don't allow for the conclusion that predgap performs worse.

In [50]:
prediction_gap_by_random_sampling(big_x, f, banzhaf_top_four, num_iter = 1000)


NameError: name 'perturbed_columns' is not defined

In [ ]:
prediction_gap_by_random_sampling(big_x, f, single_abs_predgap_top_four, num_iter = 1000)


In the calculations above the standard deviation was 0.1, below it was 1.

In [ ]:
prediction_gap_by_random_sampling(big_x, f, banzhaf_top_four, num_iter = 200)

In [ ]:
prediction_gap_by_random_sampling(big_x, f, single_abs_predgap_top_four, num_iter = 200)

## PGU results, 4 least important features are perturbed

In [ ]:
prediction_gap_by_random_sampling(big_x, f, banzhaf_bottom, num_iter = 100)


In [ ]:
prediction_gap_by_random_sampling(big_x, f, single_abs_predgap_bottom, num_iter = 100)

In [ ]:
banzhaf_bottom

In the calculations above the standard deviation was 0.1, below it was 1.

In [ ]:
prediction_gap_by_random_sampling(big_x, f, banzhaf_bottom, num_iter = 200)

In [ ]:
prediction_gap_by_random_sampling(big_x, f, single_abs_predgap_bottom, num_iter = 200)